# **Fine-tuning for Image Classification with 🤗 Transformers in TensorFlow**

This notebook shows how to fine-tune any pretrained Vision model for Image Classification on a custom dataset. The idea is to add a randomly initialized classification head on top of a pre-trained encoder, and fine-tune the model altogether on a labeled dataset.

## `tf.data` and `tensorflow_datasets`

This notebook leverages [`tf.data`](https://www.tensorflow.org/guide/data_performance) and [`tensorflow_datasets`](https://www.tensorflow.org/datasets) (`tfds` for short) loading and preparing a custom dataset (namely, [`tf_flowers`](https://www.tensorflow.org/datasets/catalog/tf_flowers) in this tutorial). If you want to use a dataset that is not present in `tfds` then you can follow [this guide](https://www.tensorflow.org/tutorials/load_data/images). 

## Any model

This notebook is built to run on any image classification dataset with any vision model checkpoint from the [Model Hub](https://huggingface.co/) as long as that model has a version with an image classification head, such as:
* [TFViTForImageClassification](https://huggingface.co/docs/transformers/model_doc/vit#transformers.TFViTForImageClassification),
* [TFSwinForImageClassification](https://huggingface.co/docs/transformers/main/en/model_doc/swin#transformers.TFSwinForImageClassification),
* [TFConvNextForImageClassification](https://huggingface.co/docs/transformers/model_doc/convnext#transformers.TFConvNextForImageClassification) 

In short, any model supported by [TFAutoModelForImageClassification](https://huggingface.co/docs/transformers/model_doc/auto#transformers.TFAutoModelForImageClassification).


## Data augmentation

This notebook leverages the ops offered by the [`tf.image`](https://www.tensorflow.org/api_docs/python/tf/image/) module for data augmentation. We will soon be adding data augmentation support with libraries like [`albumentations`](https://albumentations.ai/).


---

Depending on the model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those two parameters, then the rest of the notebook should run smoothly.

In this notebook, we'll fine-tune from the https://huggingface.co/microsoft/swin-tiny-patch4-window7-224 checkpoint, but note that there are many, many more available on the [hub](https://huggingface.co/models?other=vision).